# Comment juger de la qualité d’une tâche de classification ?

Lorsque *TreeTagger* attribue une étiquette morpho-syntaxique à un mot-forme, il effectue une tâche de classification qui repose sur un modèle probabiliste. Mais comment savoir si le résultat est pertinent ou s’il est complètement à côté de la plaque ?

Les métriques retenues vont dépendre de la tâche réalisée et, dans cet exercice, nous reconnaissons trois grandes familles :

- La **classification binaire**, qui répond à la question : « L’observation appartient-elle à la classe cible ou non ? »
- La **classification multi-classes**, qui répond à la question : « Parmi toutes, à quelle classe l’observation appartient-elle ? »
- La **classification multi-étiquettes**, qui répond à la question : « L’observation appartient-elle à plusieurs classes ? »

Dans le cas de *TreeTagger*, nous sommes en présence d’une classification multi-classes qui, à réfléchir de plus près, se révèle être un ensemble de classificateurs binaires pour lesquels nous pouvons mobiliser des métriques bien connues.

## Quoi et comment évaluer ?

L’évaluation d’une tâche de classification ne s’effectue pas directement sur les prédictions d’un algorithme. Elle nécessite de les comparer avec des résultats connus. Par exemple, nous savons qu’un linguiste a évalué tel mot comme un adjectif et nous cherchons à savoir si notre modèle de langage a permis d’effectuer le même classement.

Pour cela, il convient de disposer avant tout d’un jeu de données état de l’art, un *gold standard*, de préférence validé manuellement par des spécialistes du domaine. En linguistique, il en existe pour toutes les langues. Citons rapidement le *French Treebank* pour le français, le *Penn Treebank* ou le *Brown Corpus* pour l’anglais, le *Eindhoven-corpus* pour le néerlandais, le *Sinica Treebank* pour le mandarin, le *Qatar Arabic Corpus* pour l’arabe moderne etc.

Lorsque l’on souhaite entraîner un algorithme sur un corpus de référence, il est coutume de séparer les données en deux parties : un jeu pour l’entraînement constitué de 80 % et un autre pour le test qui contiendra les 20 % restants. À ce stade, il est important de garder en mémoire que 100 % des données sont validées mais que le modèle entraîné n’en connaît qu’une partie.

Une fois l’entraînement achevé, on utilise le modèle sur le jeu de test, en lui cachant les étiquettes réelles, avant de comparer les résultats. On ne se contentera jamais de vérifier la correspondance entre les étiquettes, mais, en mobilisant d’autres mesures, on essaiera de déterminer si les classements sont le fait du hasard ou d’un biais quelconque.

```mermaid
flowchart TD
  A[(Corpus)]:::formal --> B{{Séparation des données}}:::formal
  B -->|80%| C[/Jeu d'entraînement\]:::academic
  B -->|20%| D[/Jeu de test\]:::academic
classDef formal fill:#fff,stroke:#D68738,color:#D68738
classDef academic fill:#032B4F,stroke:#032B4F,color:#fff
```

## Mesures générales de la performance

### L’exactitude (*accuracy*)

La toute première leçon est de ne jamais oublier qu’un très mauvais classificateur peut obtenir des résultats stupéfiants. Prenons l’exemple d’un jeu de données factice qui comporte une centaine d’images étiquetées manuellement selon deux modalités : *chat* ou *pas chat* :

```csv
image_001.jpg	chat
image_002.jpg	pas chat
image_003.jpg	pas chat
…
image_100.jpg	chat
```

Pour générer un tel fichier, nous pouvons recourir à l’utilitaire `awk`, puis éditer au hasard cinq lignes pour leur attribuer l’étiquette *chat* :

In [ ]:
! seq -f "image_%03g.jpg" 1 100 \
  | awk '{print $0 "\tpas chat"}' > ./data/catalog.tsv

Un comptage rapide, avec par exemple la commande `grep`, nous apprend que seulement cinq de ces images représentent des chats :

In [ ]:
! grep -c '\tchat' ./data/catalog.tsv

Construisons maintenant un classificateur très naïf qui attribue systématiquement à toutes les images la classe *pas chat* :

In [ ]:
! for i in $(seq -w 1 100); do \
    echo -e "image_${i}.jpg\tpas chat"; \
done > ./data/output.tsv

Si nous vérifions la correspondance entre les prédictions de notre algorithme et la réalité du jeu de données, nous obtenons un taux d’exactitude très satisfaisant :

In [ ]:
! paste ./data/catalog.tsv ./data/output.tsv \
  | awk -F'\t' '$2 == $4 {count++} END {print count}'

**95 % !** Un taux à faire pâlir les diseuses de bonne aventure, non ? Pour cette raison, même s’il s’agit d’une première mesure incontournable, on ne se satisfera jamais du score d’exactitude, quitte même à s’en méfier dès que les jeux de données sont asymétriques.

### Une matrice de confusion

La matrice de confusion repose sur un principe simple : compter le nombre de fois où les observations ont été bien ou mal étiquetées. Elle révèle ainsi quatre informations essentielles :

- les vrais positifs (*true positive*), le classificateur a repéré qu’il s’agissait d’un chat ;
- les vrais négatifs (*true negative*), le classificateur a repéré qu’il ne s’agissait pas d’un chat ;
- les faux positifs (*false positive*), le classificateur a cru qu’il s’agissait d’un chat ;
- les faux négatifs (*false negative*), le classificateur aurait dû voir qu’il s’agissait d’un chat.

Appliquons ces principes à notre exemple plus haut. Chaque ligne correspond à une classe réelle et chaque colonne à une classe prédite avec, sur la première ligne, la classe positive et, sur la seconde, la classe négative :

| Classe réelle / prédite | Chat   | Pas Chat |
|-------------------------|--------|----------|
| **Chat**                | TP (0) | FN (5)   |
| **Pas Chat**            | FP (0) | TN (95)  |

**Explication des résultats :**

- **TP (0) :** L’algorithme n’a jamais correctement identifié une image de chat.
- **FN (5) :** L’algorithme a raté le classement des cinq images de chats en leur attribuant l’étiquette "pas chat".
- **FP (0) :** L’algorithme n’a jamais prédit à tort qu’une image était celle d’un chat.
- **TN (95) :** L’algorithme a correctement prédit 95 fois qu’il n’y avait pas de chat sur l’image.

La matrice de confusion apporte des raffinements au score d’exactitude déterminé plus haut. Si de prime abord on pouvait se réjouir d’un taux de réussite de 95 %, on se rend compte ici des manques de notre classificateur naïf.

Évaluer à partir de données brutes n’étant jamais évident, la matrice de confusion est le point de départ pour déterminer plusieurs métriques : la précision, la sensibilité (ou rappel) et le score $F_1$.

## Évaluer un classificateur binaire

Dans le cas d’un projet basé sur un classificateur binaire, on fera toujours appel à une matrice de confusion à partir de laquelle on obtiendra d’autres métriques plus parlantes.

### La précision

La précision (*precision*) s’intéresse à l’exactitude des prédictions positives. Elle se calcule en effectuant le rapport entre les vrais positifs – les fois où le classificateur a correctement étiqueté la classe positive – et la somme de toutes les prédictions positives, incluant donc les fois où le classificateur s’est trompé. La formule vaut :

$$
\text{precision} = \frac{TP}{TP + FP}
$$

Le score de précision s’exprime sur un intervalle $[0, 1]$ où 1 signifie que toutes les prédictions positives faites par le modèle sont correctes (il n’y a aucun faux positif) et 0 que toutes les prédictions positives faites par le modèle sont incorrectes (aucun vrai positif).

En résumé, un score élevé signifie que le modèle commet très peu d’erreurs en prédisant la classe positive alors qu’un score faible révèle qu’il prédit souvent à tort la classe positive.

Pour notre exemple :

$$
\text{precision} = \frac{0}{0 + 0}
$$

Dans cette situation, nous obtenons une précision indéfinie (division par 0) que nous pouvons à juste titre interpréter comme une précision nulle.

### Le rappel

Précision et rappel sont deux métriques complémentaires essentielles pour évaluer un classificateur binaire. Si la précision nous renseigne sur la fiabilité des prédictions positives, le rappel (*recall*), ou sensibilité, détermine la capacité du modèle à identifier correctement toutes les instances de la classe positive. Il s’obtient avec la formule :

$$
\text{recall} = \frac{TP}{TP + FN}
$$

Le rappel s’exprime également sur un intervalle $[0, 1]$ pour indiquer la capacité du modèle à trouver toutes les instances positives dans les données. Un modèle dont le rappel est proche de 1 détecte presque toutes les instances positives alors qu'un modèle avec un rappel faible laisse passer beaucoup d'instances positives sans les détecter.

Pour notre exemple :

$$
\text{recall} = \frac{0}{0 + 5} = 0
$$

Notre classificateur n’a réussi à identifier aucune instance de la classe positive, ratant ainsi toutes les occurrences qui auraient dû être détectées.

### Le score $F_1$

Le score $F_1$ effectue la moyenne harmonique entre la précision et le rappel selon la formule :

$$
F_1 = 2 \cdot \frac{\text{precision} \times \text{recall}}{\text{precision} + \text{recall}}
$$

La moyenne harmonique donnant plus d’importance aux valeurs faibles, un classificateur ne pourra être bien noté par cette métrique que si sa précision et son rappel sont élevés. Cette métrique accorde un poids égal aux deux mesures, ce qui en fait un bon compromis lorsqu’aucune n’est à privilégier. Toujours établi dans un intervalle $[0,1]$, le score $F_1$ peut soit refléter un bon équilibre entre précision et rappel dans le cas où il se rapproche de 1, soit indiquer des problèmes à bien les équilibrer s’il est plutôt proche de 0. Signalons l’existence de variantes comme le score $F_\beta$ qui permet de donner plus de poids au rappel ($\beta > 1$) ou à la précision ($\beta < 1$) selon les besoins spécifiques du problème.

Pour notre exemple, nous obtenons :

$$
F_1 = 2 \cdot \frac{0 \times 0}{0 + 0}
$$

Encore une fois, nous pouvons interpréter le $F_1$ indéfini comme étant nul.

### Autres mesures de la performance

Il existe d’autres métriques d’évaluation que nous n’aborderons pas en détail ici. Certaines, comme la *Log Loss*, sont particulièrement adaptées aux modèles qui produisent des probabilités plutôt que des prédictions binaires directes. D’autres, comme les courbes ROC ou Précision-Rappel, permettent de visualiser les performances du modèle pour différents seuils de décision.

Ces métriques incluent notamment :

- La *Log Loss* (ou *Cross-entropy*) qui évalue la qualité des probabilités prédites.
- La courbe ROC (*Receiver Operating Characteristic*) et son aire sous la courbe (AUC pour *Area Under the Curve*) qui visualisent le compromis entre vrais et faux positifs.
- La courbe Précision-Rappel qui est particulièrement pertinente pour les jeux de données déséquilibrés.

## Application à un cas réel : système de détection de fraude bancaire

Pour illustrer l’importance de ces métriques, prenons l’exemple d’une banque qui souhaite détecter automatiquement les transactions frauduleuses. Dans ce contexte :

- La classe positive représente les transactions frauduleuses ;
- la classe négative représente les transactions légitimes.

Analysons les différentes métriques pour comprendre leur pertinence business.

### La précision dans ce contexte

Une précision de 0,8 signifie que 80 % des transactions signalées comme frauduleuses par le système le sont réellement. Les 20 % restants sont des fausses alertes qui correspondent à des transactions légitimes incorrectement classées comme frauduleuses.

Une faible précision implique que les équipes anti-fraude perdent du temps à investiguer des transactions légitimes et que les clients subissent des blocages injustifiés de leurs transactions. En revanche, une précision très élevée peut signifier que le système est trop conservateur : il applique un blocage systématique en présence d’une transaction incertaine, quitte à mécontenter les client·es à l’origine d’opérations parfaitement en règle. Cela pourrait également être le signe d’un surentraînement : le système ne repère que des cas évidents alors qu’il manque les cas plus subtils de fraude.

### Le rappel dans ce contexte

Un rappel de 0,9 indique que le système détecte 90 % des fraudes réelles. Cependant, 10 % des transactions frauduleuses passent inaperçues.

Un faible rappel est particulièrement problématique car chaque fraude non détectée représente une perte financière directe pour la banque ou ses clients. À l’inverse, un rappel très élevé s’obtient souvent au prix d’une précision dégradée.

### Le compromis précision-rappel

Dans ce cas d’usage, le coût d’une fraude non détectée (faux négatif) est généralement plus élevé que celui d’une fausse alerte (faux positif) :

- Un faux positif entraîne une investigation inutile et potentiellement un mécontentement client ;
- un faux négatif conduit à une perte financière directe qui peut être conséquente.

Pour cette raison, la banque pourrait choisir de privilégier le rappel par rapport à la précision, acceptant ainsi plus de fausses alertes pour minimiser le risque de rater des fraudes réelles.

### Le score $F_1$ comme métrique de synthèse

Le score $F_1$ permet d’évaluer rapidement si le système maintient un bon équilibre. Dans notre exemple, un score $F_1$ de 0,85 indiquerait que le système parvient à maintenir à la fois une bonne précision et un bon rappel, répondant ainsi efficacement aux objectifs business :

- Détecter un maximum de fraudes ;
- minimiser la charge de travail liée aux fausses alertes ;
- limiter les blocages injustifiés de transactions légitimes.